In [5]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AutoTokenizer, AutoModel 
import pickle

# Load TinyBERT model and tokenizer
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")
# Example input texts (batch)
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through the first two layers
model_part1 = tinybert.encoder.layer[:2]

with torch.no_grad():
    # Get the initial hidden states from the embedding layer
    hidden_states = tinybert.embeddings(inputs['input_ids'])
    
    # Pass through the first two layers
    for layer in model_part1:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('172.16.19.59', 10300))  # Adjust host and port as needed
client_socket.sendall(data)
client_socket.close()
print("Intermediate output sent to server.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


AttributeError: 'BertForSequenceClassification' object has no attribute 'encoder'

In [12]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# Example input texts (batch)
texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "The service was fantastic!",
    "Not worth the price.",
    "I would recommend this!"
]

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass through the first two layers
model_part1 = tinybert.bert.encoder.layer[:2]  # Access the encoder correctly

with torch.no_grad():
    # Get the initial hidden states from the embedding layer
    hidden_states = tinybert.bert.embeddings(inputs['input_ids'])
    
    # Pass through the first two layers
    for layer in model_part1:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

# Serialize and send the intermediate tensor to the server
data = pickle.dumps(hidden_states)

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('172.16.19.59', 10300))  # Adjust host and port as needed
    client_socket.sendall(data)
except Exception as e:
    print(f"Error sending data: {e}")
finally:
    client_socket.close()

print("Intermediate output sent to server.")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Intermediate output sent to server.


In [5]:
model_part1

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=312, out_features=312, bias=True)
        (key): Linear(in_features=312, out_features=312, bias=True)
        (value): Linear(in_features=312, out_features=312, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=312, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=312, out_features=1200, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=1200, out_features=312, bias=True)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [13]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# Example input texts (batch)
texts = [
    "I love this product!",               # Expected label: 1
    "This is the worst experience I've ever had.",  # Expected label: 0
    "The service was fantastic!",         # Expected label: 1
    "Not worth the price.",               # Expected label: 0
    "I would recommend this!"             # Expected label: 1
]

# Example true labels (adjust these based on your actual labels)
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Replace with actual labels

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == true_labels).float().mean().item()
print(f"Accuracy: {accuracy * 100:.2f}%")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 60.00%


In [ ]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Use the test set
test_texts = dataset['test']['text']
test_labels = torch.tensor(dataset['test']['label'])

# Tokenize the input texts
inputs = tokenizer(test_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == test_labels).float().mean().item()
print(f"Accuracy on IMDb test set: {accuracy * 100:.2f}%")


In [1]:
hi

NameError: name 'hi' is not defined

In [2]:
"ji"


'ji'

In [1]:
import socket
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle

# Load TinyBERT model and tokenizer from local files
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")
tokenizer = BertTokenizer.from_pretrained("./optimized_tinybert")

# IMDb test reviews and their expected labels
texts = [
    "I loved this movie! It was fantastic and well-made.",      # Expected label: 1
    "This was the worst film I have ever seen. Very disappointing.",  # Expected label: 0
    "An absolute masterpiece. The performances were outstanding.",  # Expected label: 1
    "I wouldn't recommend this to anyone. It's a waste of time.",  # Expected label: 0
    "Great storyline and excellent direction. A must-watch!",      # Expected label: 1
    "Terrible acting and a boring plot. I regret watching it."     # Expected label: 0
]

# Corresponding true labels for the reviews
true_labels = torch.tensor([1, 0, 1, 0, 1, 0])  # Replace with actual labels

# Tokenize the input texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Get model predictions
with torch.no_grad():
    outputs = tinybert(**inputs)  # Forward pass
    logits = outputs.logits  # Get the logits
    predictions = torch.argmax(logits, dim=-1)  # Get the predicted classes

# Calculate accuracy
accuracy = (predictions == true_labels).float().mean().item()
print(f"Accuracy: {accuracy * 100:.2f}%")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 50.00%
